In [0]:
%sql
-- Switch to your new catalog
USE CATALOG databricks_cata;

-- Create the 'default' schema if it's missing
CREATE SCHEMA IF NOT EXISTS bronze;

In [0]:
# ==========================================
# 1. CONFIGURATION
# ==========================================
storage_account_name = "databricksete1995"

# PATHS
# Input: Where Event Hubs dumps the raw Avro files
source_path = f"abfss://source@{storage_account_name}.dfs.core.windows.net/eh-tsunami-project/earthquake-stream"

# Output: Where you want the clean Delta Table to live
bronze_path = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/earthquake_raw"

# System Paths (Checkpoints track what has been processed)
# It is best practice to keep checkpoints inside the target container but hidden
checkpoint_path = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/_checkpoints/earthquake_load"
schema_path     = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net/_checkpoints/earthquake_schema"

table_name = "databricks_cata.bronze.bronze_earthquake"

print(f"📥 Reading from: {source_path}")
print(f"📤 Writing to:   {bronze_path}")

# ==========================================
# 2. READ STREAM (From Source Container)
# ==========================================
df_raw = (spark.readStream
    .format("cloudFiles")               # Use Auto Loader
    .option("cloudFiles.format", "avro")# Read Avro format
    .option("cloudFiles.schemaLocation", schema_path)
    .load(source_path))

# ==========================================
# 3. TRANSFORM (Make it Readable)
# ==========================================
# Event Hubs Avro wraps the real data in a binary 'Body' field.
# We cast it to STRING to make it readable JSON.
df_readable = df_raw.selectExpr(
    "cast(Body as string) as json_payload", # <--- This makes it readable
    "EnqueuedTimeUtc as ingestion_time",
    "SequenceNumber",
    "Offset"
)

# ==========================================
# 4. WRITE STREAM (To Bronze Container)
# ==========================================
query = (df_readable.writeStream
    .format("delta")                    # Write as Delta Lake (Parquet-based)
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path)
    .option("path", bronze_path)        # <--- Forces storage in 'bronze' container
    .option("mergeSchema", "true")      # Auto-adjust if columns change
    .trigger(availableNow=True)
    .table(table_name))                 # Registers table in Databricks

print(f"🚀 Pipeline started! Converting Avro to Delta...")

In [0]:
# Load the table into a DataFrame
df = spark.table("databricks_cata.bronze.bronze_earthquake")

# Display the first few rows
display(df)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW v_earthquake_complete AS
SELECT 
    -- ==========================================
    -- 1. IDENTIFIERS & GEOMETRY
    -- ==========================================
    get_json_object(json_payload, '$.id') as id,
    get_json_object(json_payload, '$.type') as feature_type,
    
    -- Geometry (Coordinates: [Longitude, Latitude, Depth])
    CAST(get_json_object(json_payload, '$.geometry.coordinates[0]') AS DECIMAL(10,4)) as longitude,
    CAST(get_json_object(json_payload, '$.geometry.coordinates[1]') AS DECIMAL(10,4)) as latitude,
    CAST(get_json_object(json_payload, '$.geometry.coordinates[2]') AS DECIMAL(10,2)) as depth,

    -- ==========================================
    -- 2. MAIN PROPERTIES
    -- ==========================================
    CAST(get_json_object(json_payload, '$.properties.mag') AS DECIMAL(4,2)) as mag,
    get_json_object(json_payload, '$.properties.magType') as magType,
    get_json_object(json_payload, '$.properties.place') as place,
    get_json_object(json_payload, '$.properties.type') as type, -- e.g., "earthquake"

    -- Time (Converted from Epoch Long Integer to Timestamp)
    CAST(get_json_object(json_payload, '$.properties.time') / 1000 AS TIMESTAMP) as time_utc,
    CAST(get_json_object(json_payload, '$.properties.updated') / 1000 AS TIMESTAMP) as updated_utc,
    CAST(get_json_object(json_payload, '$.properties.tz') AS INTEGER) as tz,

    -- ==========================================
    -- 3. ALERTS & INTENSITY
    -- ==========================================
    get_json_object(json_payload, '$.properties.alert') as alert, -- e.g., "green", "red"
    get_json_object(json_payload, '$.properties.status') as status, -- e.g., "automatic", "reviewed"
    CAST(get_json_object(json_payload, '$.properties.tsunami') AS INTEGER) as tsunami,
    CAST(get_json_object(json_payload, '$.properties.sig') AS INTEGER) as sig, -- Significance
    CAST(get_json_object(json_payload, '$.properties.felt') AS INTEGER) as felt, -- Number of reports
    CAST(get_json_object(json_payload, '$.properties.cdi') AS DECIMAL(4,1)) as cdi, -- Community intensity
    CAST(get_json_object(json_payload, '$.properties.mmi') AS DECIMAL(4,2)) as mmi, -- Instrumental intensity

    -- ==========================================
    -- 4. TECHNICAL / SENSOR DATA
    -- ==========================================
    get_json_object(json_payload, '$.properties.net') as net, -- Network (e.g., "us", "ak")
    get_json_object(json_payload, '$.properties.code') as code,
    get_json_object(json_payload, '$.properties.ids') as ids,
    get_json_object(json_payload, '$.properties.sources') as sources,
    get_json_object(json_payload, '$.properties.types') as types,
    
    CAST(get_json_object(json_payload, '$.properties.nst') AS INTEGER) as nst, -- Number of Stations
    CAST(get_json_object(json_payload, '$.properties.dmin') AS DECIMAL(10,4)) as dmin, -- Min Distance
    CAST(get_json_object(json_payload, '$.properties.rms') AS DECIMAL(10,4)) as rms, -- Root Mean Square
    CAST(get_json_object(json_payload, '$.properties.gap') AS DECIMAL(10,2)) as gap, -- Azimuthal Gap

    -- ==========================================
    -- 5. URLS
    -- ==========================================
    get_json_object(json_payload, '$.properties.url') as url,
    get_json_object(json_payload, '$.properties.detail') as detail

FROM databricks_cata.bronze.bronze_earthquake;

-- Verify the output
SELECT * FROM v_earthquake_complete ORDER BY time_utc DESC LIMIT 10;